In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
from glom import glom

### Heart Rate Signal 

In [2]:
df = pd.read_json("/Users/olumide/Documents/Dissertation/Sound Stimuli version 1/MyFitbitData version 2/OlumideOdetunde/Physical Activity/heart_rate-2022-06-15.json") #load data

# extract each value
df_bpm = df['value'].apply(lambda row:glom(row,"bpm"))
df_confidence = df["value"].apply(lambda row:glom(row, "confidence"))
df_datetime = df["dateTime"]

#combine extracted for required timeframe
df_hr = pd.concat([df_datetime, df_bpm, df_confidence], axis=1, ignore_index=True)
df_hr.columns = ["datetime","heartrate","confidence"] #rename column
df_hr.set_index("datetime",inplace=True)
df_hr = df_hr.loc["2022-06-15 18:45:01": "2022-06-15 20:00:01"]
df_hr.reset_index(inplace=True)

#print and save to local directory 
df_hr.to_csv('/Users/olumide/Documents/Dissertation/Sound Stimuli version 1/heartrate.csv')
print("signal successfully saved")
df_hr.head()

signal successfully saved


,datetime,heartrate,confidence
0,2022-06-15 18:45:05,70,3
1,2022-06-15 18:45:10,73,3
2,2022-06-15 18:45:25,75,3
3,2022-06-15 18:45:30,74,3
4,2022-06-15 18:45:35,73,3


### ECG Reading

In [3]:
df_ecg = pd.read_csv("/Users/olumide/Documents/Dissertation/Sound Stimuli version 1/MyFitbitData version 2/OlumideOdetunde/Heart/Afib ECG Readings.csv")
df_ecg.head()

,reading_id,reading_time,wire_id,result_classification,heart_rate,heart_rate_alert,firmware_version,device_app_version,hardware_version,waveform_samples
0,24acf5e0-e4f8-11ec-8080-808080808080,Sun Jun 05 17:51:35 UTC 2022,497267d1532c,NSR,63,NONE,128.6.12,2.9.0,Sense,[-32768 -32768 -32768 -32768 -32768 -3276...
1,cc22f1a0-e325-11ec-8080-808080808080,Fri Jun 03 10:13:21 UTC 2022,497267d1532c,NSR,60,NONE,128.6.12,2.9.0,Sense,[32767 32767 32767 32767 32767 32767 327...


### Skin Temperature Signal

In [4]:

df_skt = pd.read_csv("/Users/olumide/Documents/Dissertation/Sound Stimuli version 1/MyFitbitData version 2/OlumideOdetunde/Sleep/Wrist Temperature - 2022-06-15.csv")

#extract values for required timeframe
df_skt.columns = ["datetime","temperature"]
df_skt["datetime"] = pd.to_datetime(df_skt['datetime'], format="%Y-%m-%dT%H:%M:%S.%f", errors = 'coerce') #changes iso dateformat
df_skt.set_index("datetime", inplace=True)
df_skt = df_skt.loc["2022-06-15 18:41":"2022-06-15 20:00"]
df_skt.reset_index(inplace=True)

#print and save to local directory
df_skt.to_csv('/Users/olumide/Documents/Dissertation/Sound Stimuli version 1/skintemp.csv')
print("signal successfully saved")
df_skt.head()


signal successfully saved


,datetime,temperature
0,2022-06-15 18:41:00,-6.460064
1,2022-06-15 18:42:00,-6.415064
2,2022-06-15 18:43:00,-6.415064
3,2022-06-15 18:44:00,-1.945064
4,2022-06-15 18:45:00,-1.330064


### Blood Oxygen variation

In [5]:
df_bo = pd.read_csv("/Users/olumide/Documents/Dissertation/Sound Stimuli version 1/MyFitbitData version 2/OlumideOdetunde/Other/estimated_oxygen_variation-2022-06-15.csv")
df_bo["timestamp"] =pd.to_datetime(df_bo.timestamp)
df_bo.rename(columns={"timestamp":"datetime"}, inplace=True)
df_bo.set_index("datetime", inplace=True)
df_bo = df_bo.loc["2022-06-15 18:41":"2022-06-15 20:00"]
df_bo.reset_index(inplace=True)

#print and save to local directory
df_bo.to_csv('/Users/olumide/Documents/Dissertation/Sound Stimuli version 1/estimatedo2variation.csv')
print("signal successfully saved")
df_bo.tail()

,datetime,Infrared to Red Signal Ratio
46,2022-06-15 19:55:31,-27
47,2022-06-15 19:56:32,-12
48,2022-06-15 19:57:32,0
49,2022-06-15 19:58:32,0
50,2022-06-15 19:59:32,-2


### Join physiological signals

In [11]:
merge1 = df_skt.merge(df_bo,on="datetime",how="left")
merge1.head()

,datetime,temperature,Infrared to Red Signal Ratio
0,2022-06-15 18:41:00,-6.460064,NaN
1,2022-06-15 18:42:00,-6.415064,NaN
2,2022-06-15 18:43:00,-6.415064,NaN
3,2022-06-15 18:44:00,-1.945064,NaN
4,2022-06-15 18:45:00,-1.330064,NaN


## Dataset Alignment and Creation

### Load Data Labels

In [7]:
df_datalabel = pd.read_excel("/Users/olumide/Documents/Dissertation/Sound Stimuli version 1/Excel Database.xlsx")
df_datalabel.head()

,SoundID,Starttime,Emotion,Emotionrating,Endtime,UserID,Soundgroup,Soundgroupselection
0,0319_2,15/06/2022 18:51:47,Fear,5,15/06/2022 18:51:58,1,Practice,Practice Sound
1,0617_2,15/06/2022 18:52:00,Fear,4,15/06/2022 18:52:14,1,Practice,Practice Sound
2,0124_2,15/06/2022 18:52:16,Happiness,4,15/06/2022 18:52:32,1,Practice,Practice Sound
3,0533_2,15/06/2022 18:52:35,Sadness,4,15/06/2022 18:52:49,1,Practice,Practice Sound
4,0194_2,15/06/2022 18:52:50,Fear,5,15/06/2022 18:53:07,1,Practice,Practice Sound


### Aligning signals and labels


In [ ]:
df_datalabel.rename(columns={"Starttime":"datetime"}, inplace=True)
df_datalabel['datetime'] = pd.to_datetime(df_datalabel.datetime, format='%d/%m/%Y %H:%M:%S')
df_datalabel.head()

,SoundID,datetime,Emotion,Emotionrating,Endtime,UserID,Soundgroup,Soundgroupselection
0,0319_2,2022-06-15 18:51:47,Fear,5,15/06/2022 18:51:58,1,Practice,Practice Sound
1,0617_2,2022-06-15 18:52:00,Fear,4,15/06/2022 18:52:14,1,Practice,Practice Sound
2,0124_2,2022-06-15 18:52:16,Happiness,4,15/06/2022 18:52:32,1,Practice,Practice Sound
3,0533_2,2022-06-15 18:52:35,Sadness,4,15/06/2022 18:52:49,1,Practice,Practice Sound
4,0194_2,2022-06-15 18:52:50,Fear,5,15/06/2022 18:53:07,1,Practice,Practice Sound


In [ ]:
merged_dataframe = pd.merge_asof(df_datalabel, df_combined, on="datetime", 
                                 tolerance=pd.Timedelta("1s"),allow_exact_matches=True)

In [ ]:
merged_dataframe.to_csv('/Users/olumide/Documents/Dissertation/Sound Stimuli version 1/merged_df.csv')
print("merged data succesfully saved")
merged_dataframe.head()

merged data succesfully saved


,SoundID,datetime,Emotion,Emotionrating,Endtime,UserID,Soundgroup,Soundgroupselection,heartrate,confidence
0,0319_2,2022-06-15 18:51:47,Fear,5,15/06/2022 18:51:58,1,Practice,Practice Sound,NaN,NaN
1,0617_2,2022-06-15 18:52:00,Fear,4,15/06/2022 18:52:14,1,Practice,Practice Sound,76.0,2.0
2,0124_2,2022-06-15 18:52:16,Happiness,4,15/06/2022 18:52:32,1,Practice,Practice Sound,74.0,3.0
3,0533_2,2022-06-15 18:52:35,Sadness,4,15/06/2022 18:52:49,1,Practice,Practice Sound,NaN,NaN
4,0194_2,2022-06-15 18:52:50,Fear,5,15/06/2022 18:53:07,1,Practice,Practice Sound,75.0,3.0
